# Building Machine Learning Systems That Don't Suck


References:

1. [Azure ML Studio Setup](https://learn.microsoft.com/en-us/azure/machine-learning/quickstart-create-resources?view=azureml-api-2)
2. [ML Studio Compute creation](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-compute-instance?view=azureml-api-2&tabs=python)
3. [ML Studio instance pricing](https://azure.microsoft.com/en-us/pricing/details/machine-learning/#pricing)
4. [AWS-Azure Compute comparison](https://www.justaftermidnight247.com/insights/aws-to-azure-instance-mapping-for-easy-comparison/)

Also see `azure_setup.md`. This file documents initial setup steps to get you up and running


## Section 1: Introduction and Initial Setup


In [2]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

import json
import logging
import sys
from pathlib import Path

import ipytest

CODE_FOLDER = Path("code")
sys.path.extend([f"./{CODE_FOLDER}"])

DATA_FILEPATH = "penguins.csv"

ipytest.autoconfig(raise_on_error=True)

# By default, basic information about HTTP sessions (URLs, headers, etc.)
# is logged at INFO level. Detailed DEBUG level logging, including request/response
# bodies and unredacted headers, can be enabled on a client with the `logging_enable` argument.
# See full SDK logging documentation with examples in the link below.
# https://learn.microsoft.com/en-us/azure/developer/python/sdk/azure-sdk-logging
#
# To prevent these from spoiling the output of this notebook cells,
# we can change the logging  level to ERROR instead.
logging.getLogger("azure.ai.ml").setLevel(logging.WARNING)

Let's now load the workspace configuration using the sdk and create a client for later use.


In [3]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

client = MLClient.from_config(
    credential=DefaultAzureCredential(), file_path="../config.json"
)


If you are running the pipeline in Local Mode on an ARM64 machine (for example, on Apple Silicon), you will need to use a custom Docker image to train and evaluate the model. Let's create a variable indicating if we are running on an ARM64 machine.


In [4]:
# We can retrieve the architecture of the local
# computer using the `uname -m` command.
architecture = !(uname -m)

IS_ARM64_ARCHITECTURE = architecture[0] == "arm64"


In [5]:
# Creating compute
# https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-compute-instance?view=azureml-api-2&tabs=python#create

# Compute Instances need to have a unique name across the region.
# Here we create a unique name with current datetime
from azure.ai.ml.entities import ComputeInstance, AmlCompute
import datetime

# ci_basic_name = "mlschool" + datetime.datetime.now().strftime("%Y%m%d%H%M")
ci_basic_name = "mlschool"
ci_basic = ComputeInstance(
    name=ci_basic_name, size="Standard_F4s_v2", idle_time_before_shutdown_minutes="15"
)
client.begin_create_or_update(ci_basic).result()

ComputeInstance({'state': 'Running', 'last_operation': {'operation_name': 'Create', 'operation_time': '2024-04-06T11:31:37.727Z', 'operation_status': 'Succeeded', 'operation_trigger': 'User'}, 'os_image_metadata': <azure.ai.ml.entities._compute._image_metadata.ImageMetadata object at 0x000002CCD095E150>, 'services': [{'display_name': 'Jupyter', 'endpoint_uri': 'https://mlschool.centralindia.instances.azureml.ms/tree/'}, {'display_name': 'Jupyter Lab', 'endpoint_uri': 'https://mlschool.centralindia.instances.azureml.ms/lab'}], 'type': 'computeinstance', 'created_on': '2024-04-06T11:31:30.618821+0000', 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'mlschool', 'description': None, 'tags': None, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/2acef264-d285-40af-be00-8dae3516307c/resourceGroups/ml-school-rg/providers/Microsoft.MachineLearningServices/workspaces/ml-school/computes/mlschool', 'Resource__source_path': '', 'base_path': 'd:\\DataspellPro

In [11]:
# Get compute
ci_basic_state = client.compute.get(ci_basic_name)
ci_basic_state

ComputeInstance({'state': 'Stopped', 'last_operation': {'operation_name': 'Stop', 'operation_time': '2024-04-06T11:35:59.453Z', 'operation_status': 'Succeeded', 'operation_trigger': 'User'}, 'os_image_metadata': <azure.ai.ml.entities._compute._image_metadata.ImageMetadata object at 0x000002CCD04B7890>, 'services': [{'display_name': 'Jupyter', 'endpoint_uri': 'https://mlschool.centralindia.instances.azureml.ms/tree/'}, {'display_name': 'Jupyter Lab', 'endpoint_uri': 'https://mlschool.centralindia.instances.azureml.ms/lab'}], 'type': 'computeinstance', 'created_on': '2024-04-06T11:31:30.618821+0000', 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'mlschool', 'description': None, 'tags': None, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/2acef264-d285-40af-be00-8dae3516307c/resourceGroups/ml-school-rg/providers/Microsoft.MachineLearningServices/workspaces/ml-school/computes/mlschool', 'Resource__source_path': '', 'base_path': 'd:\\DataspellProje

In [25]:
from azure.ai.ml.entities import PipelineJobSettings

config = {
    "default_datastore": "penguins",
    "default_compute": ci_basic_name,
    "continue_on_step_failure": False,
    "framework_version": "2.12",
    "py_version": "py311",
}

settings = PipelineJobSettings().fromkeys(config)
print(settings.items())

None
